In [1]:
import yaml

In [2]:
config_path = open("labClass6.yaml")
config = yaml.safe_load(config_path)

In [4]:
weather_key = config['weather_key']
bucket = config['bucket']
cities = config['cities']

In [6]:
import requests
from datetime import date, timedelta

end = date.today()
start = end - timedelta(7)
dates = [str(start+timedelta(days=x)) for x in range((end-start).days)]
base_url = 'http://api.weatherapi.com/v1/'
history_api = 'history.json?' 
auth = f'key={weather_key}'

In [11]:
data = []

for city in cities:
    for date in dates:
        request_body = base_url + history_api + auth + f"&q={city}&dt={date}"
        payload = requests.get(request_body).json()
        payload['dt'] = date
        payload['city'] = city
        data.append(payload)

In [13]:
from google.cloud import storage

In [14]:
client = storage.Client()

In [15]:
gcs_bucket = client.get_bucket(bucket)

In [16]:
import json

In [18]:
for row in data:
    path = f"weather/{row['dt']}/{row['city']}/forecast.json"
    blob = gcs_bucket.blob(path)
    with blob.open(mode = 'w') as file:
        json.dump(row, file)